In [ ]:
input_path = "/content/drive/MyDrive/machine learning projects/training set/monkey pox/"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
import albumentations as A

from PIL import Image

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'test': 
    datasets.ImageFolder(input_path + 'test', data_transforms['test'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    max_score=0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
        if(max_score<epoch_acc):
          torch.save(model.state_dict(), '/content/drive/MyDrive/machine learning projects/monkey pox/weight/resnet(50).h5')
          print("model saved~")
          max_score=epoch_acc
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=50)

Epoch 1/50
----------
train loss: 0.4749, acc: 0.7831
test loss: 0.3627, acc: 0.8650
model saved~
Epoch 2/50
----------
train loss: 0.3343, acc: 0.8601
test loss: 0.2651, acc: 0.9072
model saved~
Epoch 3/50
----------
train loss: 0.2973, acc: 0.8818
test loss: 0.2553, acc: 0.9072
Epoch 4/50
----------
train loss: 0.2661, acc: 0.8871
test loss: 0.2794, acc: 0.8945
Epoch 5/50
----------
train loss: 0.2714, acc: 0.8953
test loss: 0.2411, acc: 0.8987
Epoch 6/50
----------
train loss: 0.2498, acc: 0.9005
test loss: 0.2387, acc: 0.9030
Epoch 7/50
----------
train loss: 0.2529, acc: 0.8945
test loss: 0.2480, acc: 0.9156
model saved~
Epoch 8/50
----------
train loss: 0.2285, acc: 0.9132
test loss: 0.2336, acc: 0.9114
Epoch 9/50
----------
train loss: 0.2672, acc: 0.8856
test loss: 0.2936, acc: 0.8987
Epoch 10/50
----------
train loss: 0.2525, acc: 0.8960
test loss: 0.2924, acc: 0.8903
Epoch 11/50
----------
train loss: 0.2333, acc: 0.9147
test loss: 0.2591, acc: 0.8945
Epoch 12/50
----------
t

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights=None).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/machine learning projects/monkey pox/weight/resnet(50).h5'), strict=False)

RuntimeError: ignored

In [ ]:
import os
test_mon= os.listdir("/content/drive/MyDrive/machine learning projects/training set/monkey pox/test/monkeypox")
print(test_mon)
test_oth= os.listdir("/content/drive/MyDrive/machine learning projects/training set/monkey pox/test/others")
print(test_oth)
classes = os.listdir("/content/drive/MyDrive/machine learning projects/training set/monkey pox/test")
print(classes)

In [ ]:
mon_list = [Image.open( "/content/drive/MyDrive/machine learning projects/training set/monkey pox/test/monkeypox/" + img_path) for img_path in test_mon]
oth_list = [Image.open( "/content/drive/MyDrive/machine learning projects/training set/monkey pox/test/others/" +img_path) for img_path in test_oth]
print(oth_list)

In [ ]:
validation_batch1 = torch.stack([data_transforms['test'](img).to(device)
                                for img in mon_list])
validation_batch2 = torch.stack([data_transforms['test'](img).to(device)
                                for img in oth_list])

In [ ]:
pred_logits_tensor = model(validation_batch1)
pred_logits_tensor


In [ ]:
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()
pred_probs

In [ ]:
fig, axs = plt.subplots(len(mon_list), 1, figsize=(150, 100))
for i, img in enumerate(mon_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% other, {:.0f}% monkeypox".format(100*pred_probs[i,0],
                                                            100*pred_probs[i,1]))
    ax.imshow(img)

In [ ]:
pred_logits_tensor = model(validation_batch2)
pred_logits_tensor
pred_probs = F.softmax(pred_logits_tensor, dim=2).cpu().data.numpy()
pred_probs

In [ ]:
fig, axs = plt.subplots(len(oth_list), 1, figsize=(150, 100))
for i, img in enumerate(oth_list):
    ax = axs[i]
    ax.axis('off')
    ax.set_title("{:.0f}% monkeypox, {:.0f}% other".format(100*pred_probs2[i,0],
                                                            100*pred_probs2[i,1]))
    ax.imshow(img)